In [2]:
from dis import dis

'1' + '2'
dis('1'.__add__)

TypeError: don't know how to disassemble method-wrapper objects

In [5]:
(2).__add__(3)

5

In [7]:
('2').__add__('3')

'23'

https://github.com/python/cpython/blob/main/Objects/longobject.c

```C
/* Add the absolute values of two integers. */

static PyLongObject *
x_add(PyLongObject *a, PyLongObject *b)
{
    Py_ssize_t size_a = Py_ABS(Py_SIZE(a)), size_b = Py_ABS(Py_SIZE(b));
    PyLongObject *z;
    Py_ssize_t i;
    digit carry = 0;

    /* Ensure a is the larger of the two: */
    if (size_a < size_b) {
        { PyLongObject *temp = a; a = b; b = temp; }
        { Py_ssize_t size_temp = size_a;
            size_a = size_b;
            size_b = size_temp; }
    }
    z = _PyLong_New(size_a+1);
    if (z == NULL)
        return NULL;
    for (i = 0; i < size_b; ++i) {
        carry += a->ob_digit[i] + b->ob_digit[i];
        z->ob_digit[i] = carry & PyLong_MASK;
        carry >>= PyLong_SHIFT;
    }
    for (; i < size_a; ++i) {
        carry += a->ob_digit[i];
        z->ob_digit[i] = carry & PyLong_MASK;
        carry >>= PyLong_SHIFT;
    }
    z->ob_digit[i] = carry;
    return long_normalize(z);
}
```

for `str`
```C
static PyObject *
string_add(PyObject *self, PyObject *other)
{
    if (!PyUnicode_Check(other)) {
        PyErr_Format(PyExc_TypeError,
                     "can only concatenate str (not \"%s\") to str",
                     Py_TYPE(other)->tp_name);
        return NULL;
    }
    return PyUnicode_Concat(self, other);
}
```